<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4 打造自己的 ChatGPT

學會使用 API 後, 下一步就是設計自己的 ChatGPT, 首先從簡單的文字模式開始。

**準備工作**

In [ ]:
!pip install openai
!pip install rich
import openai
from google.colab import userdata
from rich import print as pprint
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## 4-1 文字模式簡易聊天程式

設計簡易對談程式

In [ ]:
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-1106",
            messages = messages
        )
        reply = response.choices[0].message.content
    except openai.APIError as err:
        reply = f"發生錯誤\n{err.error.message}"
    return reply

In [ ]:
while True:
    msg = input("你說：")
    if not msg.strip(): break
    messages = [{"role":"user", "content":msg}]
    reply = get_reply(messages)
    print(f"ㄟ唉：{reply}\n")

## 4-2 加入聊天記錄維持聊天脈絡

把歷史紀錄加入 prompt

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat(sys_msg, user_msg):
    global hist
    hist.append({"role":"user", "content":user_msg})
    reply = get_reply(hist
                      + [{"role":"system", "content":sys_msg}])
    hist.append({"role":"assistant", "content":reply})
    hist = hist[-2 * backtrace:] # 保留新的對話
    return reply

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    reply = chat(sys_msg, msg)
    print(f"{sys_msg}:{reply}\n")
hist = []

## 4-3 串流版本的聊天程式

串流版本的聊天程式

In [ ]:
def get_reply_s(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-1106",
            messages = messages,
            stream = True
        )
        for chunk in response:
            yield chunk.choices[0].delta.content or ''
    except openai.APIError as err:
        reply = f"發生錯誤\n{err.error.message}"

In [ ]:
for reply in get_reply_s([{
    "role":"user",
    "content":"請介紹台北市"
}]):
    print(reply, end='')
print('')

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_s(sys_msg, user_msg):
    global hist
    hist.append({"role":"user", "content":user_msg})
    reply_full = ""
    for reply in get_reply_s(         # 使用串流版的函式
        hist + [{"role":"system", "content":sys_msg}]):
        reply_full += reply           # 記錄到目前為止收到的訊息
        yield reply                   # 傳回本次收到的片段訊息
    hist.append({"role":"assistant", "content":reply_full})
    hist = hist[-2 * backtrace:]       # 保留最新紀錄


In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_s(sys_msg, msg):
        print(reply, end = "")
    print('\n')
hist = []

## 4-4 儲存歷史紀錄下次繼續聊

### 掛接 Google 雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 製作復原/儲存歷史紀錄的函式

In [ ]:
import pickle

In [ ]:
def save_hist(hist, sys_msg):
    try:
        with open('/content/drive/MyDrive/hist.dat', 'wb') as f:
            db = {
                'hist': hist,
                'sys_msg': sys_msg
            }
            pickle.dump(db, f)
    except:
        # 歷史檔開啟錯誤
        print('無法寫入歷史檔')

In [ ]:
def load_hist():
    try:
        with open('/content/drive/MyDrive/hist.dat', 'rb') as f:
            db = pickle.load(f)
            return db['hist'], db['sys_msg']
    except:
        # 歷史檔不存在
        print('無法開啟歷史檔')
        return [], ''

In [ ]:
hist, sys_msg = load_hist()
sys_msg_new = input(f"你希望ㄟ唉扮演[{sys_msg}]：")
if sys_msg_new: sys_msg = sys_msg_new
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_s(sys_msg, msg):
        print(reply, end = "")
    print('\n')
save_hist(hist, sys_msg)